In [4]:
import torch.nn as nn
from torch.nn import functional as F
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np 
import torch.optim as optim
import torch 
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader
import gc
import torch.utils.data as D

In [2]:
class Lstm(nn.Module):
	def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, cuda, drop_prob=0.5):
		super(Lstm, self).__init__()
		# Variables
		self.device = torch.device("cuda" if cuda else "cpu")		
		self.cuda = cuda
		self.output_size = output_size
		self.n_layers = n_layers
		self.hidden_dim = hidden_dim
		self.batch_size = 50
		# initiate layers
		self.embedding = nn.Embedding(vocab_size,embedding_dim)
		self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, batch_first=True)
		self.lin1 = nn.Linear(hidden_dim, hidden_dim) 		
		self.lin2 = nn.Linear(hidden_dim, output_size)

	# return the initial hidden state of the lstm layer (hidden state and cell state) 	
	def init_hidden(self, batch_size):
		if self.cuda:
			print("Returning cuda hidden state") 
			return (torch.randn(self.n_layers, batch_size, self.hidden_dim).cuda(), torch.randn(self.n_layers, batch_size, self.hidden_dim).cuda())
		else:
			return (torch.randn(self.n_layers, batch_size, self.hidden_dim), torch.randn(self.n_layers, batch_size, self.hidden_dim))

	# propagate features through netword
	def forward(self, x, hidden):	
		x = x.long()
		
		if self.cuda: 
			x = x.to(self.device)
		 
		embeds = self.embedding(x) 		
		out, hidden = self.lstm(embeds, hidden)
		# take only the last hidden_dim output of the lstm 
		out = self.lin1(out[:,-1,:]) 
		out = self.lin2(out)
		return out, hidden

In [5]:
class MyDataset(D.Dataset):
    def __init__(self, x_tensor, y_tensor):
        self.x = torch.from_numpy(x_tensor)
        self.y = torch.from_numpy(y_tensor)
        
    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.x)

In [19]:
def make_data(datasets, batch_size, debug): 
    print("loading datasets")
    datasets_list = []
    pos_datasets = []
    target = []

    # load all tweet datasets and merge them into one
    for f in datasets:
        datasets_list.append(pd.read_csv("../" + f + "_clean.csv"))
        pos_datasets.append(pd.read_csv("../" + f + "_pos.csv"))
    dataset = pd.concat(datasets_list, axis=0, ignore_index=True, sort=False)
    target = dataset["affect"]
    pos_dataset = pd.concat(pos_datasets, axis=0, ignore_index=True, sort=False)

    # split data into test and training data and return
    train_x, test_x, train_y, test_y = train_test_split(pos_dataset, target, test_size=0.2)

    # make data loaders
    train_data = MyDataset(train_x.to_numpy(), train_y.to_numpy())
    test_data = MyDataset(test_x.to_numpy(), test_y.to_numpy())
    train_loader = DataLoader(dataset=train_data, batch_size=batch_size)
    test_loader = DataLoader(dataset=test_data, batch_size=1)
    
    if debug: 
        dataset = dataset.iloc[:10]
        target = target[:10]
        train_x, test_x, train_y, test_y = train_test_split(dataset, target, test_size=0.8)
        train_data = MyDataset(train_x.to_numpy(), train_y.to_numpy())
        train_loader = DataLoader(dataset=train_data, batch_size=batch_size)
        test_loader = DataLoader(dataset=train_data, batch_size=1)
    return train_loader, test_loader 
    
    
    #return Variable(torch.from_numpy(train_x)), Variable(torch.from_numpy(test_x)), Variable(torch.from_numpy(train_y)), Variable(torch.from_numpy(test_y))



In [7]:
def log(summary, file):
    log = open(file, "a")
    log.write(summary)
    log.close()
    print(summary)

In [20]:
# create variables 
print("creating variables")
tweet_dataset = ["crowdflower"]
act_function = torch.sigmoid
criterion = nn.MSELoss()
cuda = torch.cuda.is_available()
train_loader, test_loader = train_loader_emotion_full, test_loader_emotion_full = make_data(tweet_dataset, 25, debug=True)

for index, (inputs, targets) in enumerate(train_loader):
    print(index, inputs)

creating variables
loading datasets


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, int64, int32, int16, int8, uint8, and bool.